# Qwen3-TTS 파인튜닝 - Debi Voice (0.6B v2)

**사전 준비:**
- Google Drive에 `debi_tts_data` 폴더 업로드
- 폴더 구조: `debi_tts_data/debi_finetune.jsonl` + `debi_tts_data/audio/*.wav`
- 런타임 유형: A100 GPU

**0.6B v2 설정:**
- flash_attention_2 사용 (A100 최적화)
- 초저학습률 (1e-6) - 모델 파괴 방지
- batch_size 8 - 학습 안정성 향상
- num_epochs 10
- 학습 후 전체 모델 저장 + HuggingFace 업로드 포함

## 셀 1: 환경 설정

In [ ]:
!apt-get install -y sox
!pip install -q soundfile librosa tqdm huggingface_hub
!pip install flash-attn --no-build-isolation
!git clone https://github.com/QwenLM/Qwen3-TTS.git /content/Qwen3-TTS-repo
%cd /content/Qwen3-TTS-repo
!pip install -e .
print("환경 설정 완료! (flash_attn 포함)")

## 셀 2: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 이전 모델 삭제 (공간 확보)
!rm -rf /content/drive/MyDrive/debi_model_0.6b
print("드라이브 마운트 완료!")

## 셀 3: 학습 코드 패치 (tensorboard만 제거)

In [ ]:
# tensorboard 제거 + 0.6B text_projection 패치
sft_path = "/content/Qwen3-TTS-repo/finetuning/sft_12hz.py"

with open(sft_path, 'r') as f:
    code = f.read()

# tensorboard -> None (Colab에서 에러 방지)
code = code.replace('log_with="tensorboard"', 'log_with=None')

# 0.6B 호환: text_projection 추가 (Issue #39)
# 1.7B는 text_embedding(2048) == codec_embedding(2048)이라 문제없지만
# 0.6B는 text_embedding(2048) != codec_embedding(1024)이므로 projection 필요
code = code.replace(
    'input_text_embedding = model.talker.model.text_embedding(input_text_ids) * text_embedding_mask',
    'input_text_embedding = model.talker.text_projection(model.talker.model.text_embedding(input_text_ids)) * text_embedding_mask'
)

with open(sft_path, 'w') as f:
    f.write(code)

print("sft_12hz.py 패치 완료!")
print("변경사항: log_with=None, text_projection 추가 (0.6B 호환)")

## 셀 4: 모델 다운로드 (config 수정 없음)

In [ ]:
from huggingface_hub import snapshot_download

# 0.6B Base 모델 다운로드 (추론 속도 개선용)
model_path = snapshot_download(
    "Qwen/Qwen3-TTS-12Hz-0.6B-Base", 
    local_dir="/content/qwen3_tts_model"
)

print(f"모델 준비 완료: {model_path}")
print("flash_attention_2 설정 유지")

## 셀 5: 오디오 24kHz 변환

In [ ]:
import os, librosa, soundfile as sf
from tqdm import tqdm

AUDIO_DIR = "/content/drive/MyDrive/debi_tts_data/audio"
OUTPUT_DIR = "/content/audio_24k"
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]
skipped = []
for f in tqdm(files):
    try:
        audio, _ = librosa.load(os.path.join(AUDIO_DIR, f), sr=24000)
        sf.write(os.path.join(OUTPUT_DIR, f), audio, 24000)
    except:
        skipped.append(f)

print(f"완료: {len(files)-len(skipped)}개, 스킵: {len(skipped)}개")

## 셀 6: JSONL 경로 업데이트

In [ ]:
import json, os

with open("/content/drive/MyDrive/debi_tts_data/debi_finetune.jsonl", 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

valid_files = set(os.listdir("/content/audio_24k"))

# 모든 샘플에 동일한 ref_audio 사용 (공식 문서 권장)
REF_AUDIO = "/content/audio_24k/Debi_airSupply_2_01.wav"

filtered = []
for item in data:
    filename = item['audio'].split('/')[-1]
    if filename in valid_files:
        item['audio'] = f"/content/audio_24k/{filename}"
        item['ref_audio'] = REF_AUDIO
        filtered.append(item)

with open("/content/debi_24k.jsonl", 'w', encoding='utf-8') as f:
    for item in filtered:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"JSONL 생성: {len(filtered)}개")
print(f"ref_audio (모든 샘플 동일): {REF_AUDIO}")

## 셀 7: 데이터 전처리 (토큰화)

In [ ]:
!python /content/Qwen3-TTS-repo/finetuning/prepare_data.py \
    --device cuda:0 \
    --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz \
    --input_jsonl /content/debi_24k.jsonl \
    --output_jsonl /content/debi_tokenized.jsonl

## 셀 8: 파인튜닝 실행 (안정화 설정)

In [ ]:
!python /content/Qwen3-TTS-repo/finetuning/sft_12hz.py \
    --init_model_path /content/qwen3_tts_model \
    --train_jsonl /content/debi_tokenized.jsonl \
    --output_model_path /content/drive/MyDrive/debi_model_0.6b \
    --batch_size 8 \
    --lr 1e-6 \
    --num_epochs 10 \
    --speaker_name debi

## 셀 9: 파인튜닝 모델 테스트

In [ ]:
import torch
import soundfile as sf
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

device = "cuda:0"
tts = Qwen3TTSModel.from_pretrained(
    "/content/drive/MyDrive/debi_model_0.6b/checkpoint-epoch-9",
    device_map=device,
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

print("Speakers:", tts.get_supported_speakers())

wavs, sr = tts.generate_custom_voice(
    text="안녕하세요, 데비입니다! 오늘도 좋은 하루 보내세요.",
    speaker="debi",
)

display(Audio(wavs[0], rate=sr))
sf.write("/content/debi_test.wav", wavs[0], sr)
print("파인튜닝 모델 테스트 완료!")

## 셀 10: 전체 모델 저장 + HuggingFace 업로드

테스트 확인 후 실행. 세션 종료 전에 반드시 실행할 것!

In [ ]:
# 세션 종료 전에 반드시 실행!
# tts 변수가 셀 9에서 이미 로드된 상태여야 함

from huggingface_hub import HfApi, login
login()

# 전체 모델 저장 (내부 HuggingFace 모델의 save_pretrained 사용)
save_path = "/content/debi_light_full"
tts.model.save_pretrained(save_path)
print(f"전체 모델 저장 완료: {save_path}")

# 파일 확인
import os
for f in sorted(os.listdir(save_path)):
    size = os.path.getsize(f"{save_path}/{f}")
    print(f"  {f}: {size/1024/1024:.1f}MB")

# config.json 수정: base -> custom_voice, speaker 등록
import json
config_path = f"{save_path}/config.json"
with open(config_path, 'r') as f:
    config = json.load(f)
config["tts_model_type"] = "custom_voice"
config["talker_config"]["spk_id"] = {"debi": 3000}
config["talker_config"]["spk_is_dialect"] = {"debi": False}
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)
print("config.json 수정 완료 (custom_voice + speaker)")

# HuggingFace 업로드
api = HfApi()
api.create_repo("2R4mi/qwen3-tts-debi-light", exist_ok=True)
api.upload_folder(
    folder_path=save_path,
    repo_id="2R4mi/qwen3-tts-debi-light",
    ignore_patterns=[".cache/**"],
)
print("HuggingFace 업로드 완료!")